## Load Credentials from a JSON

In [1]:
import json
import tweepy
import re
from collections import Counter
import time
from decouple import config
#!pip install trio
import trio

In [2]:
#Load Twitter Credentials File
TWITTER_AUTH = tweepy.OAuthHandler(config('TWITTER_CONSUMER_KEY'),config('TWITTER_CONSUMER_SECRET'))

TWITTER_AUTH.set_access_token(config('TWITTER_ACCESS_TOKEN'),config('TWITTER_ACCESS_TOKEN_SECRET'))

TWITTER = tweepy.API(TWITTER_AUTH)


"""with open('twitter_credentials.json') as cred_data:
    info = json.load(cred_data)
    TWITTER_AUTH = tweepy.OAuthHandler(info['CONSUMER_KEY'], info['CONSUMER_SECRET'])
    TWITTER_AUTH.set_access_token(info['ACCESS_KEY'],info['ACCESS_SECRET'])

TWITTER = tweepy.API(TWITTER_AUTH)""";


## Function 1 - First Layer Interactions

In [3]:
async def first_async_get_user_interactions(search, output, next_query):
    # Initialize search for the specific user.
    # Get the user's tweets from their timeline. 
    # Cycle through all the tweets' text and join it into a mega-string.
    # Do some standardizing and replacing.
    # Strip away everything except usernames, into a string. 
    # Make a list of the counts, and take the top (X) most common people. 
    # Creates a list of the `top` people. No duplicates 
    # Tuple the results of the search & output together.
    # Make search inputs for the next level of crawling. 
    twitter_user = TWITTER.get_user(search)
    tweets = twitter_user.timeline(
                count=200,
                exclude_replies=False,
                include_rts=True,
                tweet_mode='extended'
    )
    b = [ i.full_text for i in tweets ]
    b = " ".join(b)
    b = b.lower()
    b = b.replace(search, "")
    out = re.findall(r'(?<=^|(?<=[^a-zA-Z0-9-_\.]))@([A-Za-z]+[A-Za-z0-9-_]+)',b)
    top = Counter(out).most_common(20)
    interactions = []
    for interaction_count in top:
        interactions += ([interaction_count[0]] * interaction_count[1])    
    tweet_data = [(search, i) for i in interactions]
    
    output.extend(tweet_data)
    
    next_target_users = [person[0] for person in top]
    
    next_query.extend(next_target_users)

## Function 2 - Second Layer Interactions

In [4]:
async def second_async_get_user_interactions(search, output):
    # Initialize search for the specific user. 
    twitter_user = TWITTER.get_user(search)
    # Get the user's tweets from their timeline. 
    
    tweets = twitter_user.timeline(
                count=200,
                exclude_replies=False,
                include_rts=True,
                tweet_mode='extended'
    )
    # Cycle through all the tweets' text and join it into a mega-string.
    b = [ i.full_text for i in tweets ]
    b = " ".join(b)
    # Do some standardizing and replacing.
    b = b.lower()
    b = b.replace(search, "")
    # Strip away everything except usernames, into a string. 
    out = re.findall(r'(?<=^|(?<=[^a-zA-Z0-9-_\.]))@([A-Za-z]+[A-Za-z0-9-_]+)',b)
        
    # Tuple the results of the search & output together.
    tweet_data = [(search, i) for i in out]
    output = output.extend(tweet_data)

## Trio Async Tests

## 1. Drill down. (No 1st Layer Grouping) 

In [5]:
async def interaction_chain(first_user):
    data = []
    next_users = []
    start_time = time.time()
    
    # Level 1 Run. 
    await first_async_get_user_interactions(first_user, data, next_users)

    print("----------Entering 2nd level. Time so far:", time.time() - start_time)
    display(next_users)
    
    # Level 2 Run. 
    loop_num = 1
    async with trio.open_nursery() as nursery:
        for user in next_users:
            print("Loop # ", loop_num, " . Time so far:", time.time() - start_time)
            loop_num +=1
            nursery.start_soon(second_async_get_user_interactions, user, data)
            
    print("Total time:", time.time() - start_time)
    return data

In [6]:
data = trio.run(interaction_chain, 'austen')

----------Entering 2nd level. Time so far: 0.9805309772491455


['lambdaschool',
 'honestduane',
 'rjnance',
 'johnnewcastle2',
 'slouischarles',
 'bitcoinbingbong',
 'webdevmason',
 'ohmaar1',
 'danoherrin',
 'popeyethe13th',
 'merrill_r',
 'paulg',
 'diffequation',
 'noahrawlings',
 'erostribe',
 'matfitz',
 'eghenry',
 'antonjw',
 'thisismissem',
 'andrewrharvey']

Loop #  1  . Time so far: 0.9997141361236572
Loop #  2  . Time so far: 0.9999129772186279
Loop #  3  . Time so far: 1.000166893005371
Loop #  4  . Time so far: 1.0005152225494385
Loop #  5  . Time so far: 1.0007750988006592
Loop #  6  . Time so far: 1.000920057296753
Loop #  7  . Time so far: 1.0010082721710205
Loop #  8  . Time so far: 1.0011091232299805
Loop #  9  . Time so far: 1.0012328624725342
Loop #  10  . Time so far: 1.0013740062713623
Loop #  11  . Time so far: 1.0015521049499512
Loop #  12  . Time so far: 1.0016980171203613
Loop #  13  . Time so far: 1.0017831325531006
Loop #  14  . Time so far: 1.001866102218628
Loop #  15  . Time so far: 1.0021371841430664
Loop #  16  . Time so far: 1.00235915184021
Loop #  17  . Time so far: 1.002521276473999
Loop #  18  . Time so far: 1.0026960372924805
Loop #  19  . Time so far: 1.0028512477874756
Loop #  20  . Time so far: 1.002985954284668
Total time: 22.991114139556885


In [7]:
data

[('austen', 'lambdaschool'),
 ('austen', 'lambdaschool'),
 ('austen', 'lambdaschool'),
 ('austen', 'lambdaschool'),
 ('austen', 'lambdaschool'),
 ('austen', 'lambdaschool'),
 ('austen', 'lambdaschool'),
 ('austen', 'lambdaschool'),
 ('austen', 'lambdaschool'),
 ('austen', 'lambdaschool'),
 ('austen', 'lambdaschool'),
 ('austen', 'lambdaschool'),
 ('austen', 'lambdaschool'),
 ('austen', 'lambdaschool'),
 ('austen', 'lambdaschool'),
 ('austen', 'lambdaschool'),
 ('austen', 'lambdaschool'),
 ('austen', 'lambdaschool'),
 ('austen', 'lambdaschool'),
 ('austen', 'lambdaschool'),
 ('austen', 'lambdaschool'),
 ('austen', 'lambdaschool'),
 ('austen', 'lambdaschool'),
 ('austen', 'lambdaschool'),
 ('austen', 'lambdaschool'),
 ('austen', 'lambdaschool'),
 ('austen', 'lambdaschool'),
 ('austen', 'lambdaschool'),
 ('austen', 'lambdaschool'),
 ('austen', 'lambdaschool'),
 ('austen', 'lambdaschool'),
 ('austen', 'lambdaschool'),
 ('austen', 'lambdaschool'),
 ('austen', 'lambdaschool'),
 ('austen', 'h